In [1]:
import pandas as pd

In [2]:
insta_meta = pd.read_csv('recipes.csv', sep=';')
tag_columns = [i for i in insta_meta.columns if 'tag' in i]

In [11]:
tag_columns = [i for i in insta_meta.columns if 'tag' in i]

In [23]:
insta_meta["test"] = insta_meta[tag_columns].sum(axis=1)

In [25]:
insta_meta["test"]

14

In [17]:
test = insta_meta.set_index('photo_id')[tag_columns].stack()

In [16]:
test

23958

In [5]:
insta_meta.melt(id_vars=["tag_healthy"])

,tag_healthy,variable,value
0,0,photo_id,CMPELQqh_nh
1,0,photo_id,CMPEHqppR-t
2,0,photo_id,CMPEBY9Buko
3,0,photo_id,CMPEAvhF9i1
4,0,photo_id,CMPD73jJssJ
...,...,...,...
1204683,0,tag_strawberry,0
1204684,0,tag_strawberry,0
1204685,0,tag_strawberry,0
1204686,0,tag_strawberry,0


In [ ]:
insta_meta['test'] = pd.qcut(insta_meta['likes'], 10, precision=0)

In [4]:
from sklearn.datasets import make_multilabel_classification
X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=3, n_labels=2, random_state=1)

In [5]:
X

array([[ 3.,  3.,  6., ..., 11.,  1.,  3.],
       [ 7.,  6.,  4., ...,  4.,  6.,  4.],
       [ 5.,  5., 13., ..., 11.,  4.,  2.],
       ...,
       [ 4.,  3.,  6., ..., 11.,  1.,  3.],
       [ 2.,  4., 12., ...,  8.,  1.,  2.],
       [ 3.,  3.,  3., ...,  3.,  3.,  5.]])

In [26]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

# get the dataset
def get_dataset():
    X, y = make_multilabel_classification(n_samples=1000, n_features=10, n_classes=3, n_labels=2, random_state=1)
    return X, y

# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # make a prediction on the test set
        yhat = model.predict(X_test)
        # round probabilities to class labels
        yhat = yhat.round()
        # calculate accuracy
        acc = accuracy_score(y_test, yhat)
        # store result
        print('>%.3f' % acc)
        results.append(acc)
    return results

# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

ModuleNotFoundError: No module named 'keras'